In [77]:
import string
import spacy
spacy_nlp = spacy.load("en_core_web_sm")
from spacy.lang.en.stop_words import STOP_WORDS

import utility_functions as utils
import importlib
import pandas as pd
from itertools import chain
import numpy as np
from tqdm import tqdm
import json
from scipy.sparse import dok_matrix
import matplotlib.pyplot as plt

from contextualized_topic_models.models.ctm import ZeroShotTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation

# Octis is the library which can use different implemented topic modelling techniques
from octis.preprocessing.preprocessing import Preprocessing
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.models.LDA import LDA
from octis.models.CTM import CTM
from octis.models.ETM import ETM
from octis.models.NeuralLDA import NeuralLDA
from octis.models.NMF import NMF

importlib.reload(utils)
data = './preprocessed_df.pkl'

In [ ]:
df = pd.read_pickle(data)
df[['Artist', 'Song', 'Tokens', 'Lyrics', 'Coast']].head()

In [ ]:
# Create the corpus required by OCTIS to build up the dataset
with open('corpus.tsv', 'w', encoding='utf-8') as file:
    for lyrics in df['Lyrics']:
        if pd.notna(lyrics):
            file.write(lyrics + '\n')

In [ ]:
# Flatten all tokens to create a single list of words
vocab = set(chain.from_iterable(df['Tokens'].tolist()))

# Save as vocabulary.json
with open("./vocabulary.json", 'w') as f:
    json.dump(list(vocab), f)

In [ ]:
# Initialize the document-term matrix
num_docs = len(df)
num_terms = len(vocab)
doc_term_matrix = dok_matrix((num_docs, num_terms), dtype=np.int32)

# Build the matrix
token_to_index = {word: idx for idx, word in enumerate(vocab)}

for doc_idx, tokens in enumerate(df['Tokens']):
    for token in tokens:
        if token in token_to_index:
            word_idx = token_to_index[token]
            doc_term_matrix[doc_idx, word_idx] += 1

# Convert the matrix to a sparse format JSON
sparse_matrix = []
for (doc_idx, word_idx), freq in doc_term_matrix.items():
    sparse_matrix.append([doc_idx, word_idx, int(freq)])

# Save as doc_term_matrix.json
with open("./doc_term_matrix.json", 'w') as f:
    json.dump(sparse_matrix, f)

In [ ]:
# Initialize preprocessing
preprocessor = Preprocessing(
    vocabulary=None,
    max_features=None,
    remove_punctuation=True,
    punctuation=string.punctuation,
    lemmatize=True,
    min_chars=2,
    min_words_docs=0,
    save_original_indexes=True,
    min_df=0.05,
    max_df=0.8,
    split=True
)

dataset = preprocessor.preprocess_dataset(documents_path="./corpus.tsv")

In [67]:
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.evaluation_metrics.diversity_metrics import TopicDiversity

# Initialize the coherence and diversity metrics
coherence_cv = Coherence(topk=10, measure='c_v')
coherence_umass = Coherence(topk=10, measure='u_mass')
topic_diversity = TopicDiversity(topk=10)

In [84]:
from itertools import product

# Define hyperparameter grids for each model
param_grids = {
    'LDA': {
        'num_topics': [2, 3, 4, 5],
        'iterations': [500, 1000],
        'random_state': [42]
    },
    'CTM': {
        'num_topics': [2, 3, 4, 5],
        'num_epochs': [5, 10],
    },
    'ETM': {
        'num_topics': [2, 3, 4, 5],
        'num_epochs': [50, 100],
    },
    'NeuralLDA': {
        'num_topics': [2, 3, 4, 5],
        'num_epochs': [50, 100],
        'lr': [2e-3, 1e-3],
    },
    'NMF': {
        'num_topics': [2, 3, 4, 5],
        'random_state': [42]
    }
}

def evaluate_coherence(model_output):
    # Initialize the coherence metric
    coherence_cv = Coherence(topk=10, measure='c_v')

    # Calculate and return coherence score
    coherence_score = coherence_cv.score(model_output)
    return coherence_score

def parameter_search(model_name, dataset, param_grid):
    param_combinations = list(product(*param_grid.values()))
    best_score = -float('inf')
    best_params = None
    best_model_output = None

    # Use tqdm to track progress in parameter search
    for params in tqdm(param_combinations, desc=f"Searching {model_name} Params"):
        # Create parameter dict
        param_dict = dict(zip(param_grid.keys(), params))

        # Initialize and train the model based on model name and params
        if model_name == 'LDA':
            model = LDA(**param_dict)
        elif model_name == 'CTM':
            model = CTM(**param_dict)
        elif model_name == 'ETM':
            model = ETM(**param_dict)
        elif model_name == 'NeuralLDA':
            model = NeuralLDA(**param_dict)
        elif model_name == 'NMF':
            model = NMF(**param_dict)

        # Train the model
        model_output = model.train_model(dataset)

        # Evaluate the model (e.g., using coherence score)
        score = evaluate_coherence(model_output)

        if score > best_score:
            best_score = score
            best_params = param_dict
            best_model_output = model_output

    return best_model_output, best_params, best_score

In [85]:
best_models = {}

# Perform hyperparameter search for each model with progress bars
for model_name, param_grid in tqdm(param_grids.items(), desc="Overall Model Parameter Search"):
    best_output, best_params, best_score = parameter_search(model_name, dataset, param_grid)
    best_models[model_name] = {'output': best_output, 'params': best_params, 'score': best_score}
    print(f"Best {model_name} Params: {best_params} with Score: {best_score}")

Overall Model Parameter Search:  20%|██        | 1/5 [00:53<03:34, 53.74s/it]

Best LDA Params: {'num_topics': 2, 'iterations': 500, 'random_state': 42} with Score: 0.6017872659552621



Overall Model Parameter Search:  40%|████      | 2/5 [01:08<01:32, 30.91s/it]

Best CTM Params: {'num_topics': 3, 'num_epochs': 5} with Score: 0.6173094858246226



Searching ETM Params:   0%|          | 0/8 [00:00<?, ?it/s]

...

Epoch: [1/100]	Samples: [956/95600]	Train Loss: 1179.2978066161088	Time: 0:00:00.028482
Epoch: [1/100]	Samples: [205/20500]	Validation Loss: 1152.2553925304878	Time: 0:00:00.002975
Epoch: [2/100]	Samples: [1912/95600]	Train Loss: 1174.1072543475418	Time: 0:00:00.026631
Epoch: [2/100]	Samples: [205/20500]	Validation Loss: 1152.8133431783538	Time: 0:00:00.002697
Epoch: [3/100]	Samples: [2868/95600]	Train Loss: 1172.6026739016736	Time: 0:00:00.025888
Epoch: [3/100]	Samples: [205/20500]	Validation Loss: 1154.071169969512	Time: 0:00:00.002614
Epoch: [4/100]	Samples: [3824/95600]	Train Loss: 1164.665320998954	Time: 0:00:00.030206
Epoch: [4/100]	Samples: [205/20500]	Validation Loss: 1152.7749666539635	Time: 0:00:00.002397
Epoch: [5/100]	Samples: [4780/95600]	Train Loss: 1161.8813496992677	Time: 0:00:00.024623
Epoch: [5/100]	Samples: [205/20500]	Validation Loss: 1147.6921303353658	Time: 0:00:00.002566
Epoch: [6/100]	Samples: [5736/95600]	Train Loss: 1167.299776085251	Time: 0:00:00.026402
Epoch


Overall Model Parameter Search:  80%|████████  | 4/5 [02:25<00:36, 36.21s/it]

Best NeuralLDA Params: {'num_topics': 2, 'num_epochs': 50, 'lr': 0.001} with Score: 0.776061086940576



Overall Model Parameter Search: 100%|██████████| 5/5 [02:32<00:00, 30.49s/it]

Best NMF Params: {'num_topics': 2, 'random_state': 42} with Score: 0.6035502327204885


In [86]:
# Summarize results
import pandas as pd

summary = []
for model_name, model_info in best_models.items():
    summary.append({
        'Model': model_name,
        'Best_Params': model_info['params'],
        'Best_Score': model_info['score']
    })

summary_df = pd.DataFrame(summary)
print(summary_df)

       Model                                        Best_Params  Best_Score
0        LDA  {'num_topics': 2, 'iterations': 500, 'random_s...    0.601787
1        CTM                 {'num_topics': 3, 'num_epochs': 5}    0.617309
2        ETM                {'num_topics': 5, 'num_epochs': 50}    0.639376
3  NeuralLDA   {'num_topics': 2, 'num_epochs': 50, 'lr': 0.001}    0.776061
4        NMF              {'num_topics': 2, 'random_state': 42}    0.603550


In [95]:
best_models['LDA']

{'output': {'topic-word-matrix': array([[0.00185909, 0.00224993, 0.00613317, ..., 0.00109303, 0.00014323,
          0.00078758],
         [0.00183788, 0.00327841, 0.00513837, ..., 0.00229301, 0.00082292,
          0.00063022]], dtype=float32),
  'topics': [['nigga',
    'make',
    'fuck',
    'man',
    'shit',
    'let',
    'one',
    'want',
    'time',
    'come'],
   ['nigga',
    'want',
    'say',
    'see',
    'one',
    'make',
    'back',
    'come',
    'shit',
    'take']],
  'topic-document-matrix': array([[0.99390554, 0.97734255, 0.49285874, ..., 0.6869083 , 0.07091396,
          0.02274054],
         [0.00609445, 0.02265745, 0.50714129, ..., 0.3130917 , 0.92908597,
          0.97725952]]),
  'test-topic-document-matrix': array([[0.        , 0.74635935, 0.99336243, 0.10644579, 0.7301811 ,
          0.08416618, 0.91168082, 0.35630509, 0.23246025, 0.82655567,
          0.98141748, 0.6418463 , 0.48036799, 0.03879594, 0.61559248,
          0.79487765, 0.21062225, 0.29875869

In [96]:
best_models['CTM']

{'output': {'topics': [['double',
    'somethin',
    'hear',
    'thing',
    'world',
    'whatever',
    'stress',
    'bass',
    'ho',
    'battle'],
   ['take',
    'well',
    'little',
    'as',
    'tell',
    'guy',
    'another',
    'brother',
    'nothing',
    'wit'],
   ['clean',
    'another',
    'huh',
    'bass',
    'great',
    'blue',
    'flow',
    'makin',
    'ghetto',
    'cube']],
  'topic-document-matrix': array([[0.08450763, 0.317814  , 0.33435184, ..., 0.25211883, 0.08910767,
          0.22139151],
         [0.33204313, 0.16490646, 0.07378573, ..., 0.12000985, 0.31651188,
          0.18019159],
         [0.58344924, 0.51727953, 0.59186239, ..., 0.62787131, 0.59438041,
          0.59841692]]),
  'topic-word-matrix': array([[ 0.0170679 ,  0.03369807, -0.04947468, ..., -0.12947851,
          -0.1389453 , -0.06324163],
         [-0.02002003, -0.06443809, -0.00666221, ..., -0.022369  ,
          -0.01695826,  0.00541135],
         [-0.08495021, -0.09038385, -0